In [16]:
# https://cloud.google.com/ai-platform/training/docs/training-jobs#gcloud

# Submit & Execute Notebook on GCP

## required authentication

In [20]:
# service_account_keyfile = "<>.json"
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_keyfile

In [19]:
# !gcloud auth activate-service-account --key-file $service_account_keyfile
!gcloud config set account minesh.1291@gmail.com

Updated property [core/account].


## download kaggle data and push to gcs

In [31]:
# !mv v4/kaggle.json ~/.kaggle/kaggle.json

In [33]:
# !kaggle competitions download lish-moa 

In [48]:
!date -Idate
!gsutil cp ./lish-moa.zip gs://{BUCKET_NAME}/inputs/

2020-11-12
Copying file://./lish-moa.zip [Content-Type=application/zip]...
- [1 files][ 64.7 MiB/ 64.7 MiB]                                                
Operation completed over 1 objects/64.7 MiB.                                     


In [21]:
!gsutil cp ./pca-vs-keras-autoencoder.ipynb gs://{BUCKET_NAME}/inputs/

Copying file://./pca-vs-keras-autoencoder.ipynb [Content-Type=application/octet-stream]...
/ [1 files][ 45.0 KiB/ 45.0 KiB]                                                
Operation completed over 1 objects/45.0 KiB.                                     


## Model Name & Version

In [1]:
MODEL_NAME = "moa"
MODEL_VERSION = "v1"
PKG_VERSION = "1.0"

PROJECT_ID = "my-kaggle-competitions"
BUCKET_NAME = "mkc_transfer/ubuntu-vm-1/Minesh/Projects/2_my_kaggle/competitions/7_2020_moa"
REGION = "us-central1"

data_archive = "lish-moa.zip"
notebook_name = "pca-vs-keras-autoencoder.ipynb" # toy example

MODEL_VERSION_NAME = MODEL_NAME+"_"+MODEL_VERSION
MODEL_VERSION_NAME


'moa_v1'

In [18]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


## make folder & files

- trainer.train.py        : Train using Train Data, Notebook and Copy result to GCP Storage 


## packaging

In [51]:
%%writefile ./setup.py
from setuptools import setup, find_packages

REQUIRED_PACKAGES = []

setup(name="trainer",
      packages=["trainer"],
      package_dir={'trainer':'trainer'},
      version="1.0",
      install_requires=REQUIRED_PACKAGES,
      
      # include_package_data=True,
      # #package_data={'trainer': ['trainer/*.yml']},
      # data_files = [('trainer',['trainer/fname.yml','trainer/fname2.yml'])]
     )

Overwriting ./setup.py


In [52]:
!python setup.py sdist --formats=gztar

running sdist
running egg_info
writing trainer.egg-info/PKG-INFO
writing dependency_links to trainer.egg-info/dependency_links.txt
writing top-level names to trainer.egg-info/top_level.txt
package init file 'trainer/__init__.py' not found (or not a regular file)
reading manifest file 'trainer.egg-info/SOURCES.txt'
writing manifest file 'trainer.egg-info/SOURCES.txt'

running check


creating trainer-1.0
creating trainer-1.0/trainer.egg-info
copying files to trainer-1.0...
copying setup.py -> trainer-1.0
copying trainer.egg-info/PKG-INFO -> trainer-1.0/trainer.egg-info
copying trainer.egg-info/SOURCES.txt -> trainer-1.0/trainer.egg-info
copying trainer.egg-info/dependency_links.txt -> trainer-1.0/trainer.egg-info
copying trainer.egg-info/top_level.txt -> trainer-1.0/trainer.egg-info
Writing trainer-1.0/setup.cfg
Creating tar archive
removing 'trainer-1.0' (and everything under it)


## test local train - check for errors in script

In [80]:
# !~/anaconda3/bin/pip install google-cloud-storage

In [65]:
!mkdir outputdir

In [19]:
# %%time
# !/home/minesh_1291/anaconda3/bin/python -m trainer.train \
#   --gcs_data_path ./{data_archive} \
#   --gcs_model_path gs://{BUCKET_NAME}/{MODEL_VERSION_NAME}/outputdir \
#   --gcs_notebook_path "pca-vs-keras-autoencoder.ipynb"

## submission 

In [8]:
! gcloud ai-platform jobs submit training {MODEL_VERSION_NAME}_training_$(date +"%Y%m%d_%H%M%S") \
  --job-dir gs://{BUCKET_NAME}/$MODEL_NAME/train_dirs/$(date -Idate) \
  --package-path ./trainer \
  --module-name trainer.train \
  --project $PROJECT_ID \
  --region $REGION \
  --runtime-version 2.2 \
  --python-version 3.7 \
  --scale-tier CUSTOM \
  --master-machine-type n1-standard-4 \
  -- \
  --gcs_data_path gs://{BUCKET_NAME}/inputs/{data_archive} \
  --gcs_notebook_path gs://{BUCKET_NAME}/inputs/{notebook_name} \
  --gcs_model_path gs://{BUCKET_NAME}/{MODEL_VERSION_NAME}/outputdir

                

Job [moa_v1_training_20201112_212344] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe moa_v1_training_20201112_212344

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs moa_v1_training_20201112_212344
jobId: moa_v1_training_20201112_212344
state: QUEUED


# Retrive Outputs and Resulting Notebook

In [4]:
!gsutil cp -r gs://{BUCKET_NAME}/{MODEL_VERSION_NAME}/outputdir/* ./outputdir/

Copying gs://mkc_transfer/ubuntu-vm-1/Minesh/Projects/2_my_kaggle/competitions/7_2020_moa/moa_v1/outputdir/pca_result.jpg...
Copying gs://mkc_transfer/ubuntu-vm-1/Minesh/Projects/2_my_kaggle/competitions/7_2020_moa/moa_v1/outputdir/resulting_nb_pca-vs-keras-autoencoder.ipynb...
/ [2 files][ 78.6 KiB/ 78.6 KiB]                                                
Operation completed over 2 objects/78.6 KiB.                                     


In [8]:
!rm -R .ipynb_checkpoints